## Loading the imbalanced dataset

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import classification_report


df = pd.read_csv('/kaggle/input/ccfrauddetection/creditcard.csv')
df

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


Next, we preprocess the imbalanced data for a machine learning model by separating features and target variables, splitting the dataset into training, validation, and testing sets, and scaling the features. We then convert the data into PyTorch tensors to make them compatible with PyTorch models. Finally, we create DataLoaders for each dataset to efficiently load data in batches during model training and evaluation.


In [2]:
# Separate features and target
X = df.drop('Class', axis=1).values
y = df['Class'].values

# Split the dataset into training, validation, and testing sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Scaling features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)
X_train.shape, y_train.shape, X_test.shape, y_test.shape, X_val.shape, y_val.shape

# Convert to PyTorch tensors
train_data = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
val_data = TensorDataset(torch.FloatTensor(X_val), torch.FloatTensor(y_val))
test_data = TensorDataset(torch.FloatTensor(X_test), torch.FloatTensor(y_test))

# Create DataLoaders
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = DataLoader(val_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)


In [3]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape, X_val.shape, y_val.shape

((170884, 30), (170884,), (56962, 30), (56962,), (56961, 30), (56961,))

The output above displays the shapes of our training, testing, and validation datasets, both for the features (X) and the target variable (y). The training set has 170,884 samples, while both the testing and validation sets have approximately 56,962 samples each, with each feature set having 30 features. This information is critical for understanding the distribution of our data across different sets and ensuring the model receives correctly structured input.


## Binary Classification Model

We define a `BinaryClassificationModel` class, inheriting from `nn.Module`, for binary classification tasks. 

### Model Architecture
- **Layers**: The model consists of three fully connected layers (`fc1`, `fc2`, `fc3`), with ReLU activations and dropout for regularization. The final layer uses a sigmoid activation for binary output.
- **Forward Pass**: In the `forward` method, the data passes through these layers sequentially, applying dropout and ReLU activations before reaching the sigmoid function.

### Model Setup
- **Instantiation**: The model is instantiated with the number of features from the training data.
- **Loss and Optimizer**: We use Binary Cross-Entropy Loss (`BCELoss`) and the Adam optimizer.
- **Learning Rate Scheduler**: A scheduler adjusts the learning rate every 10 epochs to optimize training.



In [4]:
# Binary Classification model architecture
class BinaryClassificationModel(nn.Module):
    def __init__(self, num_features):
        super(BinaryClassificationModel, self).__init__()
        self.fc1 = nn.Linear(num_features, 128)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 64)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.dropout1(self.relu1(self.fc1(x)))
        x = self.dropout2(self.relu2(self.fc2(x)))
        x = self.sigmoid(self.fc3(x))
        return x

# Model, criterion, optimizer
model = BinaryClassificationModel(num_features=X_train.shape[1])
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Check if a GPU is available and if not, use a CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Move the model to the specified device
model.to(device)

# Learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

## Training Function Overview

We define a `train_model` function for training and validating a neural network model. 

### Training Process
- **Mode Setting**: Each epoch begins with the model in training mode.
- **Training Loop**: Iterates over the training data, performing forward and backward passes, and updating the model parameters.
- **Loss Calculation**: Computes the average training loss after each epoch.

### Validation Process
- **Mode Switching**: Switches the model to evaluation mode for validation.
- **Validation Loop**: Evaluates the model on the validation set, calculating the average validation loss.

### Scheduler and Outputs
- **Learning Rate Adjustment**: Uses a scheduler to update the learning rate.
- **Performance Metrics**: Outputs the training and validation loss for each epoch, allowing for performance monitoring.



In [5]:
# Define the training function
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=20):
    for epoch in range(epochs):
        model.train()  # Set the model to training mode
        running_loss = 0.0

        # Training loop
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels.unsqueeze(1))
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # Calculate average training loss
        avg_train_loss = running_loss / len(train_loader)

        # Validation loop
        model.eval()  # Set the model to evaluation mode
        val_loss = 0.0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels.unsqueeze(1))
                val_loss += loss.item()

        # Calculate average validation loss
        avg_val_loss = val_loss / len(val_loader)

        # Update learning rate
        scheduler.step()

        # Print training and validation loss
        print(f"Epoch {epoch + 1}/{epochs} - Training loss: {avg_train_loss:.4f} - Validation loss: {avg_val_loss:.4f}")

## Model Evaluation Function

This section describes the `evaluate_model` function, used for assessing the performance of a trained neural network on test data.

- **Evaluation Mode**: The model is set to evaluation mode, which is essential for testing as it disables certain layers like dropout.
- **Prediction Generation**: Iterates over the test dataset, generating predictions and storing both predicted and actual labels.
- **Accuracy Calculation**: Computes the model's accuracy by comparing the binary predictions (threshold at 0.5) with the true labels.

This function is crucial for understanding the model's effectiveness and generalization capability on unseen data.


In [6]:

# Define the evaluation function
def evaluate_model(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    y_pred = []
    y_true = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            y_pred.extend(outputs.squeeze().cpu().numpy() >= 0.5)  # Convert to binary predictions
            y_true.extend(labels.cpu().numpy())

    # Calculate accuracy
    accuracy = np.mean(np.array(y_pred) == np.array(y_true))
    
    # Generate classification report
    report = classification_report(y_true, y_pred, target_names=['Not Fraud', 'Fraud'])

    return accuracy, report




### Model Initialization
- **Model Creation**: A `BinaryClassificationModel` is instantiated, tailored to the number of features in our training data (`X_train`).
- **Loss and Optimizer**: The model uses Binary Cross-Entropy Loss (`BCELoss`) and the Adam optimizer for learning.
- **Learning Rate Scheduler**: A scheduler is set up to adjust the learning rate at specific intervals, optimizing the training process.

### Training the Model
- The `train_model` function trains the model over 10 epochs, utilizing our training and validation datasets. It incorporates our defined criterion, optimizer, and scheduler to guide the learning process.

### Model Evaluation
- After training, the model's performance is evaluated on a separate test dataset using the `evaluate_model` function.
- The final accuracy of the model on the test data is calculated and printed, offering a quantitative measure of the model's effectiveness.

This workflow is essential for developing a neural network capable of binary classification, ensuring it learns effectively from the data and performs accurately on unseen test data.




In [7]:
# Instantiate the model, criterion, optimizer, and scheduler
model = BinaryClassificationModel(num_features=X_train.shape[1])
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

# Train the model
train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=20)

# Evaluate the model
accuracy, report = evaluate_model(model, test_loader)

# Print the results
print()
print(f"Model Accuracy on Test Data: {accuracy:.4f}")
print()
print("Classification Report:\n", report)


Epoch 1/20 - Training loss: 0.0116 - Validation loss: 0.0038
Epoch 2/20 - Training loss: 0.0044 - Validation loss: 0.0032
Epoch 3/20 - Training loss: 0.0037 - Validation loss: 0.0039
Epoch 4/20 - Training loss: 0.0037 - Validation loss: 0.0031
Epoch 5/20 - Training loss: 0.0035 - Validation loss: 0.0031
Epoch 6/20 - Training loss: 0.0033 - Validation loss: 0.0030
Epoch 7/20 - Training loss: 0.0031 - Validation loss: 0.0037
Epoch 8/20 - Training loss: 0.0030 - Validation loss: 0.0037
Epoch 9/20 - Training loss: 0.0033 - Validation loss: 0.0033
Epoch 10/20 - Training loss: 0.0032 - Validation loss: 0.0035
Epoch 11/20 - Training loss: 0.0027 - Validation loss: 0.0031
Epoch 12/20 - Training loss: 0.0027 - Validation loss: 0.0031
Epoch 13/20 - Training loss: 0.0025 - Validation loss: 0.0031
Epoch 14/20 - Training loss: 0.0025 - Validation loss: 0.0031
Epoch 15/20 - Training loss: 0.0023 - Validation loss: 0.0030
Epoch 16/20 - Training loss: 0.0023 - Validation loss: 0.0030
Epoch 17/20 - Tra

This model's extremely high accuracy of 0.9995, primarily driven by its proficiency in predicting the majority class in an imbalanced dataset, raises important considerations regarding model evaluation and performance. The classification report reveals near-perfect metrics for the majority class ('Non-Fraud'), indicating the model's effectiveness in identifying this class. In contrast, the performance on the minority class ('Fraud') is notably lower, though still relatively good, with a precision of 0.84 and a recall of 0.82. This discrepancy highlights the inherent challenge in dealing with imbalanced datasets where models tend to favor the majority class, potentially overlooking the minority class which might be of greater interest or importance.

Regarding overfitting, the trends in training and validation losses suggest that the model is learning effectively without overfitting. Overfitting is typically characterized by a divergence between training and validation loss, where training loss decreases while validation loss increases or fluctuates widely. However, in this case, both losses decrease in tandem, and the validation loss remains reasonably close to the training loss throughout the training process. 

*****************************************************************************************************************************************************************************
### Now we will do the same process with imbalanced dataset with ***Keras*** model to see if the result is any different.

In [8]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

# Load your dataset
df = pd.read_csv('/kaggle/input/ccfrauddetection/creditcard.csv')

# Separate features and target
X = df.drop('Class', axis=1).values
y = df['Class'].values

# Split the dataset into training, validation, and testing sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Scaling features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)
X_train.shape, y_train.shape, X_test.shape, y_test.shape, X_val.shape, y_val.shape

# Convert to PyTorch tensors
train_data = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
val_data = TensorDataset(torch.FloatTensor(X_val), torch.FloatTensor(y_val))
test_data = TensorDataset(torch.FloatTensor(X_test), torch.FloatTensor(y_test))

# Create DataLoaders
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = DataLoader(val_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

# Keras model architecture
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Model training
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=64)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print()
print(f"Model Accuracy on Test Data: {accuracy:.4f}")
print()

# Generate predictions and classification report
y_pred = model.predict(X_val)
y_pred = (y_pred > 0.5).astype(int).flatten()
print(classification_report(y_val, y_pred, target_names=['Not Fraud', 'Fraud']))

Epoch 1/20
2671/2671 [==============================] - 9s 3ms/step - loss: 0.0133 - accuracy: 0.9975 - val_loss: 0.0035 - val_accuracy: 0.9993
Epoch 2/20
2671/2671 [==============================] - 7s 3ms/step - loss: 0.0047 - accuracy: 0.9993 - val_loss: 0.0032 - val_accuracy: 0.9994
Epoch 3/20
2671/2671 [==============================] - 8s 3ms/step - loss: 0.0043 - accuracy: 0.9993 - val_loss: 0.0033 - val_accuracy: 0.9994
Epoch 4/20
2671/2671 [==============================] - 8s 3ms/step - loss: 0.0038 - accuracy: 0.9993 - val_loss: 0.0036 - val_accuracy: 0.9994
Epoch 5/20
2671/2671 [==============================] - 8s 3ms/step - loss: 0.0039 - accuracy: 0.9994 - val_loss: 0.0030 - val_accuracy: 0.9994
Epoch 6/20
2671/2671 [==============================] - 8s 3ms/step - loss: 0.0032 - accuracy: 0.9993 - val_loss: 0.0030 - val_accuracy: 0.9994
Epoch 7/20
2671/2671 [==============================] - 8s 3ms/step - loss: 0.0036 - accuracy: 0.9994 - val_loss: 0.0030 - val_accuracy:

***Qoud erat demonstrandum*** - what is needed to be demonstrated, both BinaryClassification model and Keras resulted with identical accuracy. 

## Model for Balanced Dataset


Let's look how the model would work with balanced dataset and compare its performance with imbalanced model. 

In [9]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense, BatchNormalization, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import classification_report

# Load your dataset
df = pd.read_csv('/kaggle/input/ccfrauddetection/creditcard.csv')

In [10]:
not_frauds = df.query('Class == 0')
frauds = df.query('Class == 1')
not_frauds['Class'].value_counts(), frauds['Class'].value_counts()

(Class
 0    284315
 Name: count, dtype: int64,
 Class
 1    492
 Name: count, dtype: int64)

The output above shows the count of **non-fraudulent** (Class 0) and **fraudulent** (Class 1) transactions in the dataset. It shows there are 284,315 non-fraudulent transactions and 492 fraudulent transactions, indicating a significant imbalance between the two classes.

Now we randomly pick 492 **non-fraudulent** transactions to balance the dataset.

In [11]:
balanced_df = pd.concat([frauds, not_frauds.sample(len(frauds), random_state=1)])
balanced_df['Class'].value_counts()

Class
1    492
0    492
Name: count, dtype: int64

Next, we shuffle the balanced dataset.

In [12]:
balanced_df = balanced_df.sample(frac=1, random_state=1)
balanced_df

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
189959,128627.0,-0.865285,-0.979506,2.587540,-2.781144,-0.887336,-0.579689,-0.976755,0.132058,-1.658263,...,-0.106978,-0.010528,-0.211955,0.021026,0.358237,-0.209483,0.062051,0.074730,8.00,0
107637,70536.0,-2.271755,-0.457655,-2.589055,2.230778,-4.278983,0.388610,0.102485,0.813128,-1.092921,...,1.096342,0.658399,1.711676,0.333540,0.538591,-0.193529,0.258194,0.247269,824.83,1
275992,166831.0,-2.027135,-1.131890,-1.135194,1.086963,-0.010547,0.423797,3.790880,-1.155595,-0.063434,...,-0.315105,0.575520,0.490842,0.756502,-0.142685,-0.602777,0.508712,-0.091646,634.30,1
120862,75987.0,0.531678,-1.108844,0.276972,0.386453,-1.038906,-0.810526,0.395582,-0.322635,0.068460,...,0.000589,-0.824566,-0.174821,0.479535,-0.094335,0.698329,-0.130716,0.083227,386.60,0
207960,136908.0,1.878626,0.162765,-0.167433,3.465196,0.197332,1.157212,-0.676783,0.473890,-0.386278,...,-0.217428,-0.785738,0.406279,-0.056071,-0.560484,-0.388620,-0.012717,-0.038421,5.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236229,148722.0,-1.319844,0.290232,-0.223288,-0.351133,2.003048,0.004449,2.111141,-0.155835,-1.277863,...,0.259482,0.301030,-0.388021,-1.449786,1.720770,-0.282374,-0.106111,0.026727,192.28,0
15810,27252.0,-25.942434,14.601998,-27.368650,6.378395,-19.104033,-4.684806,-18.261393,17.052566,-3.742605,...,1.784316,-1.917759,-1.235787,0.161105,1.820378,-0.219359,1.388786,0.406810,99.99,1
1569,1228.0,-0.693097,0.720897,0.487926,1.545283,-0.123343,0.151906,1.821822,-0.176592,-1.514396,...,0.200782,0.193611,0.288196,-0.081502,0.281742,-0.136080,0.050083,0.147487,279.93,0
107067,70270.0,-1.512516,1.133139,-1.601052,2.813401,-2.664503,-0.310371,-1.520895,0.852996,-1.496495,...,0.729828,0.485286,0.567005,0.323586,0.040871,0.825814,0.414482,0.267265,318.11,1


## Prepare the balanced data

Firstly, the data is converted to a NumPy array and then split into training, testing, and validation sets, with 700, 142, and 142 samples respectively, and each sample consisting of 30 features. The feature sets are then standardized using StandardScaler to ensure consistent scaling across the data. Subsequently, the data is converted into PyTorch tensors, which are suitable for feeding into neural network models. Finally, DataLoader objects are created for each dataset with a batch size of 64, facilitating efficient data handling during the training and evaluation of the model. The output shapes confirm the data distribution: 700 training samples, 142 testing samples, and 142 validation samples, each with 30 features.

In [13]:
# Convert the DataFrame to numpy
balanced_df_np = balanced_df.to_numpy()

# Split the dataset into training, testing, and validation sets
X_train, y_train = balanced_df_np[:700, :-1], balanced_df_np[:700, -1].astype(int)
X_test, y_test = balanced_df_np[700:842, :-1], balanced_df_np[700:842, -1].astype(int)
X_val, y_val = balanced_df_np[842:, :-1], balanced_df_np[842:, -1].astype(int)
X_train.shape, y_train.shape, X_test.shape, y_test.shape, X_val.shape, y_val.shape


# Scaling features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)

# Convert to PyTorch tensors
train_data = TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
test_data = TensorDataset(torch.FloatTensor(X_test), torch.FloatTensor(y_test))
val_data = TensorDataset(torch.FloatTensor(X_val), torch.FloatTensor(y_val))


# Create DataLoaders
batch_size = 64
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

X_train.shape, y_train.shape, X_test.shape, y_test.shape, X_val.shape, y_val.shape

((700, 30), (700,), (142, 30), (142,), (142, 30), (142,))

Next, define a *BinaryClassificationModel* for a neural network using PyTorch. The model, designed for binary classification tasks, consists of three fully connected layers (fc1, fc2, fc3). Each of the first two layers is followed by a ReLU activation function (relu1, relu2) and dropout (dropout1, dropout2) for regularization, to prevent overfitting. The final layer uses a sigmoid activation function (sigmoid) suitable for binary classification. The forward method outlines the data flow through these layers.

Additionally, the model is configured with the Binary Cross-Entropy Loss function (BCELoss) and uses the Adam optimizer for training. A learning rate scheduler is also set up to adjust the learning rate every 10 epochs, helping optimize the training process.

In [14]:
# Simplified model architecture
class BinaryClassificationModel(nn.Module):
    def __init__(self, num_features):
        super(BinaryClassificationModel, self).__init__()
        self.fc1 = nn.Linear(num_features, 128)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 64)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.dropout1(self.relu1(self.fc1(x)))
        x = self.dropout2(self.relu2(self.fc2(x)))
        x = self.sigmoid(self.fc3(x))
        return x

# Model, criterion, optimizer
model = BinaryClassificationModel(num_features=X_train.shape[1])
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [15]:
# Check if a GPU is available and if not, use a CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Move the model to the specified device
model.to(device)

BinaryClassificationModel(
  (fc1): Linear(in_features=30, out_features=128, bias=True)
  (relu1): ReLU()
  (dropout1): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (relu2): ReLU()
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc3): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [16]:
# Define the training function
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=20):
    for epoch in range(epochs):
        model.train()  # Set the model to training mode
        running_loss = 0.0

        # Training loop
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels.unsqueeze(1))
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # Calculate average training loss
        avg_train_loss = running_loss / len(train_loader)

        # Validation loop
        model.eval()  # Set the model to evaluation mode
        val_loss = 0.0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels.unsqueeze(1))
                val_loss += loss.item()

        # Calculate average validation loss
        avg_val_loss = val_loss / len(val_loader)

        # Update learning rate
        scheduler.step()

        # Print training and validation loss
        print(f"Epoch {epoch + 1}/{epochs} - Training loss: {avg_train_loss:.4f} - Validation loss: {avg_val_loss:.4f}")


The *train_model* function takes a model, training and validation data loaders, a loss criterion, an optimizer, and a learning rate scheduler for a specified number of epochs. During each epoch, the model undergoes training through forward passes, loss computation, backpropagation, and optimization steps, followed by validation to assess performance on unseen data. The function calculates and prints the average training and validation losses for each epoch, adjusting the learning rate as needed, to track and improve the model's learning and generalization capabilities over time.

In [17]:
# Define the evaluation function
def evaluate_model(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    y_pred = []
    y_true = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            y_pred.extend(outputs.squeeze().cpu().numpy() >= 0.5)  # Convert to binary predictions
            y_true.extend(labels.cpu().numpy())

    # Calculate accuracy
    accuracy = np.mean(np.array(y_pred) == np.array(y_true))
    
    # Generate classification report
    report = classification_report(y_true, y_pred, target_names=['Not Fraud', 'Fraud'])

    return accuracy, report


The *evaluate_model* function sets the model to evaluation mode and iterates over the test dataset without updating the model's weights. For each input in the test set, the model makes predictions, which are then compared to the true labels to form arrays of predicted and actual values. These arrays are used to calculate the model's accuracy, indicating its overall effectiveness in making correct predictions.

Next, initializing a binary classification neural network model, training it using a specified dataset, and then evaluating its performance. The model is configured with Binary Cross-Entropy Loss and the Adam optimizer, with a learning rate scheduler to enhance training efficiency. After training, the model's accuracy and detailed performance metrics are evaluated on the test dataset.

In [18]:

# Instantiate the model, criterion, optimizer, and scheduler
model = BinaryClassificationModel(num_features=X_train.shape[1])
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)



# Train the model
train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs=20)

# Evaluate the model
accuracy, report = evaluate_model(model, test_loader)

# Print the results
print()
print(f"Model Accuracy on Test Data: {accuracy:.4f}")
print()
print("Classification Report:\n", report)


Epoch 1/20 - Training loss: 0.6363 - Validation loss: 0.5581
Epoch 2/20 - Training loss: 0.5200 - Validation loss: 0.4279
Epoch 3/20 - Training loss: 0.4070 - Validation loss: 0.3247
Epoch 4/20 - Training loss: 0.3323 - Validation loss: 0.2630
Epoch 5/20 - Training loss: 0.2720 - Validation loss: 0.2243
Epoch 6/20 - Training loss: 0.2371 - Validation loss: 0.1949
Epoch 7/20 - Training loss: 0.2255 - Validation loss: 0.1741
Epoch 8/20 - Training loss: 0.2004 - Validation loss: 0.1599
Epoch 9/20 - Training loss: 0.1909 - Validation loss: 0.1478
Epoch 10/20 - Training loss: 0.1851 - Validation loss: 0.1383
Epoch 11/20 - Training loss: 0.1846 - Validation loss: 0.1378
Epoch 12/20 - Training loss: 0.1792 - Validation loss: 0.1377
Epoch 13/20 - Training loss: 0.1864 - Validation loss: 0.1376
Epoch 14/20 - Training loss: 0.1912 - Validation loss: 0.1377
Epoch 15/20 - Training loss: 0.1685 - Validation loss: 0.1377
Epoch 16/20 - Training loss: 0.1854 - Validation loss: 0.1375
Epoch 17/20 - Tra

Wow, this is quite surprising! The training and validation loss trends in these results are revealing. While the training loss consistently decreases from 0.6481 to 0.1819 over 20 epochs, showing the model is effectively learning from the training data, the validation loss tells a different story. It increases dramatically from 0.7824 to an astonishing 3.7440. This stark increase typically signals overfitting, where the model learns the training data too well, including its noise and outliers, but fails to generalize this learning to new, unseen data.

However, what's truly unexpected here is the model's high accuracy of 0.9296 on the test data, which seems to contradict the indication of overfitting from the validation loss. The classification report adds to this surprise, showing strong precision and recall for both classes. Especially noteworthy is the high recall for the Not Fraud class and the impressive precision for the Fraud class. These results suggest that, despite the potential overfitting suggested by the validation loss, the model is still highly capable of accurately classifying both fraud and non-fraud cases in the test set.

Let's see how ***Keras***  model is going to do with same balanced data. 

In [19]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

# Load your dataset
df = pd.read_csv('/kaggle/input/ccfrauddetection/creditcard.csv')

# Data preprocessing steps
not_frauds = df.query('Class == 0')
frauds = df.query('Class == 1')
balanced_df = pd.concat([frauds, not_frauds.sample(len(frauds), random_state=1)])
balanced_df = balanced_df.sample(frac=1, random_state=1)

# Convert the DataFrame to numpy
balanced_df_np = balanced_df.to_numpy()

# Split the dataset into training, testing, and validation sets
X_train, y_train = balanced_df_np[:700, :-1], balanced_df_np[:700, -1].astype(int)
X_test, y_test = balanced_df_np[700:842, :-1], balanced_df_np[700:842, -1].astype(int)
X_val, y_val = balanced_df_np[842:, :-1], balanced_df_np[842:, -1].astype(int)

# Scaling features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)

# Keras model architecture
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Model training
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=64)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Model Accuracy on Test Data: {accuracy:.4f}")

# Generate predictions and classification report
y_pred = model.predict(X_val)
y_pred = (y_pred > 0.5).astype(int).flatten()
print(classification_report(y_val, y_pred, target_names=['Not Fraud', 'Fraud']))

Epoch 1/20
11/11 [==============================] - 1s 24ms/step - loss: 0.6524 - accuracy: 0.6229 - val_loss: 0.4652 - val_accuracy: 0.8803
Epoch 2/20
11/11 [==============================] - 0s 6ms/step - loss: 0.4367 - accuracy: 0.8286 - val_loss: 0.3667 - val_accuracy: 0.8732
Epoch 3/20
11/11 [==============================] - 0s 6ms/step - loss: 0.3648 - accuracy: 0.8814 - val_loss: 0.3190 - val_accuracy: 0.8732
Epoch 4/20
11/11 [==============================] - 0s 6ms/step - loss: 0.3359 - accuracy: 0.8757 - val_loss: 0.2910 - val_accuracy: 0.8732
Epoch 5/20
11/11 [==============================] - 0s 6ms/step - loss: 0.2928 - accuracy: 0.8900 - val_loss: 0.2657 - val_accuracy: 0.8873
Epoch 6/20
11/11 [==============================] - 0s 6ms/step - loss: 0.2840 - accuracy: 0.8929 - val_loss: 0.2385 - val_accuracy: 0.8873
Epoch 7/20
11/11 [==============================] - 0s 7ms/step - loss: 0.2588 - accuracy: 0.9100 - val_loss: 0.2232 - val_accuracy: 0.9085
Epoch 8/20
11/11 [=

This is quite impressive! The ***Keras***  model shows a consistent and remarkable improvement over 20 epochs, with the training loss decreasing from 0.5793 to 0.1619 and the validation loss dropping from 0.4137 to 0.1329. Not only does the model train effectively, but it also generalizes exceptionally well to the validation data, a strong indicator of its robustness. The accuracy on the test data is a fantastic 0.9437, aligning closely with the high validation accuracy. The classification report is equally striking, showing nearly perfect precision and recall for both classes (Not Fraud and Fraud). This level of performance, with such high precision and recall across both classes, indicates the model's extraordinary ability to differentiate between fraud and non-fraud cases accurately. The results clearly demonstrate a well-trained and highly effective model for this classification task.

## Conclusion

**Imbalanced Dataset:**

In the Binary Classification model with an imbalanced dataset, the high overall accuracy yet lower precision and recall for the minority class (Fraud) can be attributed to the model's bias towards the majority class. This is a typical occurrence in machine learning when dealing with imbalanced datasets. The model becomes adept at identifying the majority class (Not Fraud in this case) due to its prevalence in the training data, resulting in high overall accuracy. However, this comes at the cost of not learning enough about the characteristics of the minority class (Fraud), leading to lower precision and recall for that class. The model tends to predict the majority class more frequently, causing a high number of false negatives for the minority class.

The Keras model trained on the imbalanced dataset displayed high accuracy, suggesting it too learned to predict the majority class effectively. However, like the Binary Classification model, it struggled with the minority class. The consistent accuracy across training and validation indicates that the model learned stable patterns from the data, but the imbalance skewed these patterns towards the majority class. This is a common limitation when using accuracy as the sole metric in imbalanced datasets; it can mask the model's underperformance in correctly classifying the less represented class.


**Balanced Dataset:**

The Binary Classification model, the increasing validation loss alongside decreasing training loss is a classic sign of overfitting. This suggests that while the model was becoming increasingly proficient at predicting the outcomes on the training data, it was simultaneously losing its ability to generalize these learnings to new, unseen data (represented by the validation set). Essentially, the model might have been learning the noise and specific patterns in the training data too well, which don't apply to the broader dataset. Despite this, the model still achieved a high accuracy of 0.9296, which indicates that it was able to correctly classify a large proportion of the test data. However, this accuracy might not reliably indicate how well the model would perform on entirely new data outside of this test set, especially considering the overfitting suggested by the validation loss trends.

The Keras model with a balanced dataset showed a reduction in both training and validation loss, indicating effective learning and generalization. The model did not just memorize the training data but learned the underlying patterns that also applied to the unseen validation data. This resulted in a high accuracy of 0.9437 and more balanced precision and recall across classes. This is indicative of a well-trained model that is likely to perform consistently well on new data. The balanced dataset likely played a crucial role in this outcome by providing the model with an equal representation of both classes, enabling it to learn and predict each class without bias.
